### This notebook
builds a graph-based embedding for a small number of random handwritten digits with multidimensional scaling.

Optional: run this before import to recompile c++ ops
```
!rm -r ../lib/cpp/temp/
!rm -r ../lib/cpp/build/
!rm -r ../lib/cpp/_bindings.so
!rm -r ../lib/cpp/bindings.py
!rm -r ../lib/cpp/lib_wrap.c
```


##### Get the dataset:

In [6]:
%load_ext autoreload
%autoreload 2
import gc
import sys
sys.path.insert(0, "..")
import time

import h5py
from IPython.display import clear_output
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
import torch, torch.nn as nn
import torch.nn.functional as F
import torchvision
from tqdm.notebook import tqdm

torch.manual_seed(42)

import lib # builds c++ binaries with swig
from lib.utils.distance_helper import compute_original_pairwise_distance, compute_pca_pairwise_distance, mds_pairwise_distance
from lib import data_loader as data_loader

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


#### Choose 1 of the 4 datasets to load

In [ ]:
dataset = 'fashion_mnist'

In [2]:
dataset = 'cifar'

In [7]:
dataset = 'fiw'

In [11]:
dataset = 'newsgroup'

#### Load dataset

In [12]:
if dataset == 'fashion_mnist':
    X, y = data_loader.get_fashion_mnist(10000)
    batch_size = 256
elif dataset == 'cifar':
    X, y = data_loader.get_cifar10(10000)
    batch_size = 256
elif dataset == 'fiw':
    X, y = data_loader.get_faces_in_wild()
    batch_size = 256
elif dataset == 'newsgroup':
    X, y = data_loader.get_newsgroup_vectors()
    batch_size = 256
else:
    raise Exception(f"Unspecified dataset: {dataset}")

#### Compute PCA and MDS compression MSE (pre-computed)

In [ ]:
distances = np.load(f'./data/{dataset}_dist.npy')
dist_pca = np.load(f'./data/{dataset}_pca.npy')
dist_mds = np.load(f'./data/{dataset}_mds.npy')

#### Else compute them using the cell below
#### Will take a few hours and huge CPU & Memory resources, even with our optimizations

In [ ]:
# distances = compute_original_pairwise_distance(X, simple=False, num_samples=len(X), temp_filename=f'{dataset}_dist')
# dist_pca = compute_pca_pairwise_distance(X, simple=False, num_samples=len(X), temp_filename=f'{dataset}_pca')
# dist_mds = mds_pairwise_distance(X)
# np.save(f'./data/{dataset}_dist.npy', distances)
# np.save(f'./data/{dataset}_pca.npy', dist_pca)
# np.save(f'./data/{dataset}_mds.npy', dist_mds)

#### Build initial graph

We initialize prodige with a full graph initialized with distances between nodes

In [ ]:
emb = lib.initalize_prodige(X, knn_edges=64, random_edges=32,verbose=True)

prune_threshold = 0.5  # increase up to 0.9 for faster convergence
loss_history, reg_history = [], []
opt = None
total_edges = emb.num_edges
num_edges_req = len(X) * 2 * 1.1  # *1.1 as buffer
stage, last_updated = 0, 0

##### Training loop

Here we minimize the MDS loss function
$$L = 1/N \sum_{i, j} (d_{orig}(x_i, x_j) - d_G(v_i, v_j))^2$$

* $d_{orig}(x_i, x_j)$ is the original distance between two vectors in $X$
* $d_G(v_i, v_j)$ is the learned graph distance between corresponding vertices in graph $G$

In [ ]:
stage, emb, opt = lib.update_learning_rate(stage, len(loss_history), emb, prune_threshold=prune_threshold)
print("Stage:", stage)
for i in tqdm(range(len(loss_history), 10000)):
    source = torch.randint(0, len(X), [batch_size])
    target = torch.randint(0, len(X), [batch_size])

    pred = emb(source, target)
    distances_ref = torch.as_tensor(distances[source, target], dtype=torch.float32)
    
    reconstruction_mse = F.mse_loss(pred['target_distances'], distances_ref)
    regularizer = emb.compute_l0_prior_penalty(4096)
    lambd = min(1, len(loss_history) / 10000.) * (emb.num_edges / total_edges) * 10.0
    # Equation 2 + log deriative trick
    loss = reconstruction_mse - pred['logp_target_paths'].mean() + lambd * regularizer
    opt.zero_grad()
    loss.backward()
    opt.step()
    loss_history.append(reconstruction_mse.item())
    reg_history.append(regularizer.item())
    
    # early stopping       
    edges_kept = np.sum(lib.check_numpy(emb.edge_adjacency_logits >= 0).astype('int64'))
    if edges_kept <= num_edges_req:
        print('Early stopping at epoch:', i)
        break

    # dynamic updating of LR/pruning  
    if len(reg_history) > 100 and i > last_updated:
        limit = round(reg_history[-30], 3)
        if stage <= 3:
            if all(round(i, 3) >= limit for i in reg_history[-30:]):
                stage, emb, opt = lib.update_learning_rate(stage, len(loss_history), emb, prune_threshold=prune_threshold)
                last_updated = i + 50
        else:
            if all(round(i, 3) >= limit for i in reg_history[-30:]):
                stage, emb, opt = lib.update_learning_rate(stage, len(loss_history), emb, 
                                                           prune_threshold=prune_threshold, decrease_lr=False)
                last_updated = i + 500

    if len(loss_history) % 100 == 0:
        clear_output(True)
        print("Stage:", stage)
        plt.figure(figsize=[15, 4])
        plt.subplot(1, 2, 1);
        plt.title('mse = %.20f' % loss_history[-1]); plt.grid()
        plt.plot(loss_history)
        plt.subplot(1, 2, 2);
        plt.title('regularizer = %0.5f' % reg_history[-1]); plt.grid()
        plt.plot(reg_history)
        plt.show()
        print(emb)

In [ ]:
dist_prodige = emb.compute_pairwise_distances()
emb.report_model_size()

In [ ]:
print("PCA:\t %.5f" % np.mean(np.square(distances - dist_pca)))
print("MDS:\t %.5f" % np.mean(np.square(distances - dist_mds)))
print("PRODIGE: %.5f" % np.mean(np.square(distances - dist_prodige)))

#### Generate PRODIGE as a NetworkX file

In [ ]:
G = lib.generate_networkx_graph(emb, y)

In [ ]:
output_file(f'{dataset}_graph.html')
p = lib.draw_networkx_graph(G, dataset, weighted=True)
save(p, f'{dataset}_graph.html')

```

```

```

```

```

```

```

```

```

```
